# Importacion de librerias

In [0]:
!pip install wbgapi
import wbgapi as wb
import pandas as pd
import numpy as np
import time
import pandas as pd
import io
import requests
from pandas import json_normalize

     |████████████████████████████████| 701 kB 5.8 MB/s 
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


# Creacion de los esquemas

In [0]:
%sql 
CREATE SCHEMA IF NOT EXISTS transformed_data;
CREATE SCHEMA IF NOT EXISTS analysis_data;

## Funcion para la extraccion de datos del Banco mundial de datos

In [0]:
#Funciones
#banco_mundial
def getDataFrameDatabank(countries_list,dataDicc):
    dataset = np.array([])
    for i in dataDicc.items():
        test = wb.data.fetch(i[0], countries_list)
        time.sleep(1)
        list_countries = []
        list_years = []
        list_data = []
        for a in test:
            list_countries.append(a['economy'])
            list_years.append((a['time'][2::]))
            list_data.append((a['value']))
        add = {'title':i[1],
            'economy':list_countries,
            'time':list_years,
            'value':list_data}
        dataset = np.append(dataset, add)

    #validation data dimentions

    for i in range(0,len(dataset)):
        print('dimention of data list: '+str(dataset[i]['title']))
        print(len(dataset[i]['value']))

    for i in range(0,len(dataset)-1):
        if (dataset[i]['time'] == dataset[i+1]['time']):
            print('is equal times')
        if (dataset[i]['economy'] == dataset[i+1]['economy']):
            print('is equal countries')

    # dataframe export
    dataframe = pd.DataFrame()
    dataframe['country'] = dataset[0]['economy'] 
    dataframe['year'] = dataset[0]['time'] 
    for i in range(0,len(dataset)):
        dataframe[dataset[i]['title']] = dataset[i]['value'] 
    return dataframe

## Lista de codigos de paises sobre los que se abordara el trabajo.

In [0]:
countries_list = ['ARG','BRA','COL','CAN','CHL','MEX','ECU','USA','URY', 'PAN','VEN',
                  'DEU','BEL','ESP','FRA','NLD','GBR','ITA','CHE','UKR', 'RUS',
                  'AGO','EGY','CMR','NGA','ZAF','ETH',
                  'CHN','JPN','KOR','MYS','IND','AFG']

## Tablas

### Esperanza de vida

In [0]:
dataEsp = {'SP.DYN.LE00.FE.IN':'Life_expectancy_at_birth_female_years',
            'SP.DYN.LE00.MA.IN':'Life_expectancy_at_birth_male_years'}

In [0]:
esperanza_vida = getDataFrameDatabank(countries_list, dataEsp)

dimention of data list: Life_expectancy_at_birth_female_years
2046
dimention of data list: Life_expectancy_at_birth_male_years
2046
is equal times
is equal countries


In [0]:
spark_esp = spark.createDataFrame(esperanza_vida)
spark_esp

Out[6]: DataFrame[country: string, year: string, Life_expectancy_at_birth_female_years: double, Life_expectancy_at_birth_male_years: double]

In [0]:
spark_esp.write.mode('overwrite').option("mergeSchema", "true").saveAsTable('transformed_data.esperanza_vida')

### Eje salud

In [0]:
dataSalud = {#Sociales
            'SH.STA.BRTC.ZS':'Births_attended_by_skilled_health_staff-%_of_total',
            #economicos
            'SH.XPD.CHEX.GD.ZS':'Current_health_expenditure-%_of_GDP',
            'SH.XPD.CHEX.PC.CD':'Current_health_expenditure_per_capita -current_US$',
            'SH.XPD.EHEX.PC.CD':'External_health_expenditure_per_capita-current_US$',
            'SH.XPD.GHED.GD.ZS':'Domestic_general_government_health_expenditure-%_of_GDP',
            'SH.XPD.GHED.PC.CD':'Domestic_general_government_health_expenditure_per_capita-current_US$',
            'SH.XPD.PVTD.PC.CD':'Domestic_private_health_expenditure_per_capita-current_US$',
            #riesgo_de_muerte
            'SH.MMR.RISK.ZS':'Lifetime_risk_of_maternal_death-%',
            'SP.DYN.LE00.FE.IN':'Life_expectancy_at_birth_female-years',
            'SP.DYN.LE00.MA.IN':'Life_expectancy_at_birth_male-years',
            #Mortalidad
            'SH.DTH.NMRT':'Number_of_neonatal_deaths',
            'SH.DYN.NMRT':'Mortality_rate_neonatal-per_1000_live_births',
            'SP.DYN.IMRT.FE.IN':'Mortality_rate_infant_female-per_1000_live_births',
            'SP.DYN.IMRT.MA.IN':'Mortality_rate_infant_male-per_1000_live_births',
            'SH.DTH.IMRT':'Number_of_infant_deaths',
            'SH.DYN.MORT.FE':'Mortality_rate_under-5_female-per 1000_live_births',
            'SH.DYN.MORT.MA':'Mortality_rate_under-5_male-per_1000_live_births',
            'SH.DTH.0509':'Number_of_deaths_ages_5-9_years',
            'SH.DTH.1014':'Number_of_deaths_ages_10-14_years',
            'SH.DTH.1519':'Number_of_deaths_ages_15-19_year',
            'SH.DTH.2024':'Number_of_deaths_ages_20-24_years',
            'SP.DYN.AMRT.FE':'Mortality_rate_adult_female-per_1000_female_adults',
            'SP.DYN.AMRT.MA':'Mortality_rate_adult_male-per_1000_male_adults',
            #maternas
            'SH.STA.MMRT.NE':'Maternal_mortality_ratio-national_estimate_per_100000_live birth',
            'SH.MMR.DTHS':'Number_of_maternal_deaths',
            #Suicidio
            'SH.STA.SUIC.FE.P5':'Suicide_mortality_rate_female-per_100000_female_population',
            'SH.STA.SUIC.MA.P5':'Suicide_mortality_rate_male-per_100000_male_population',
            #fertilidad
            'SP.ADO.TFRT':'Adolescent_fertility_rate-births_per_1000_women_ages_15-19',
            'SP.DYN.TFRT.IN':'Fertility_rate_total-births_per_woman'}


In [0]:
eje_salud = getDataFrameDatabank(countries_list, dataSalud)
eje_salud


dimention of data list: Births_attended_by_skilled_health_staff-%_of_total
2046
dimention of data list: Current_health_expenditure-%_of_GDP
2046
dimention of data list: Current_health_expenditure_per_capita -current_US$
2046
dimention of data list: External_health_expenditure_per_capita-current_US$
2046
dimention of data list: Domestic_general_government_health_expenditure-%_of_GDP
2046
dimention of data list: Domestic_general_government_health_expenditure_per_capita-current_US$
2046
dimention of data list: Domestic_private_health_expenditure_per_capita-current_US$
2046
dimention of data list: Lifetime_risk_of_maternal_death-%
2046
dimention of data list: Life_expectancy_at_birth_female-years
2046
dimention of data list: Life_expectancy_at_birth_male-years
2046
dimention of data list: Number_of_neonatal_deaths
2046
dimention of data list: Mortality_rate_neonatal-per_1000_live_births
2046
dimention of data list: Mortality_rate_infant_female-per_1000_live_births
2046
dimention of data li

,country,year,Births_attended_by_skilled_health_staff-%_of_total,Current_health_expenditure-%_of_GDP,Current_health_expenditure_per_capita -current_US$,External_health_expenditure_per_capita-current_US$,Domestic_general_government_health_expenditure-%_of_GDP,Domestic_general_government_health_expenditure_per_capita-current_US$,Domestic_private_health_expenditure_per_capita-current_US$,Lifetime_risk_of_maternal_death-%,...,Number_of_deaths_ages_15-19_year,Number_of_deaths_ages_20-24_years,Mortality_rate_adult_female-per_1000_female_adults,Mortality_rate_adult_male-per_1000_male_adults,Maternal_mortality_ratio-national_estimate_per_100000_live birth,Number_of_maternal_deaths,Suicide_mortality_rate_female-per_100000_female_population,Suicide_mortality_rate_male-per_100000_male_population,Adolescent_fertility_rate-births_per_1000_women_ages_15-19,Fertility_rate_total-births_per_woman
0,AFG,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,2020,61.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12079.0,13913.0,185.146,230.214,NaN,NaN,NaN,NaN,57.5090,4.176
2,AFG,2019,NaN,13.242202,65.806030,8.167607,1.084431,5.388990,52.249440,NaN,...,12081.0,13531.0,188.839,233.884,NaN,NaN,3.6,4.6,61.3250,4.321
3,AFG,2018,58.8,14.126743,69.998604,13.854496,0.549220,2.721409,53.422707,NaN,...,12044.0,13135.0,192.532,237.554,NaN,NaN,3.5,4.6,65.1410,4.473
4,AFG,2017,53.4,12.620817,65.706024,12.760199,0.643134,3.348260,49.597570,3.063593,...,11908.0,12631.0,198.184,240.314,NaN,7700.0,3.4,4.6,68.9570,4.633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2041,ARG,1964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,145.115,251.743,NaN,NaN,NaN,NaN,60.8324,3.068
2042,ARG,1963,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,146.321,250.957,NaN,NaN,NaN,NaN,60.9702,3.079
2043,ARG,1962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,147.528,250.171,NaN,NaN,NaN,NaN,61.1080,3.090
2044,ARG,1961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,151.879,252.620,NaN,NaN,NaN,NaN,61.2480,3.100


In [0]:
#convert the pandas dataframe to a pyspark dataframe and save it as a table:
spark_sa = spark.createDataFrame(eje_salud)



In [0]:
spark_sa.write.mode('overwrite').option("mergeSchema", "true").saveAsTable('transformed_data.eje_salud')

### Eje Economia

In [0]:
dataEcon = {'NV.SRV.TOTL.ZS': 'porcentaje_gdp_servicios',
            'NV.AGR.TOTL.ZS': 'porcentaje_gdp_agricultura',
            'NV.IND.MANF.ZS': 'porcentaje_gdp_manufactura',
            'NV.IND.TOTL.ZS': 'porcentaje_gdp_industria',
            'NY.GDP.MKTP.CD': 'gdp_usd',
            'NY.GDP.PCAP.CD': 'gdp_percapita'}

eje_economico = getDataFrameDatabank(countries_list, dataEcon)


dimention of data list: porcentaje_gdp_servicios
2046
dimention of data list: porcentaje_gdp_agricultura
2046
dimention of data list: porcentaje_gdp_manufactura
2046
dimention of data list: porcentaje_gdp_industria
2046
dimention of data list: gdp_usd
2046
dimention of data list: gdp_percapita
2046
is equal times
is equal countries
is equal times
is equal countries
is equal times
is equal countries
is equal times
is equal countries
is equal times
is equal countries


In [0]:
spark_ec = spark.createDataFrame(eje_economico)
spark_ec


Out[14]: DataFrame[country: string, year: string, porcentaje_gdp_servicios: double, porcentaje_gdp_agricultura: double, porcentaje_gdp_manufactura: double, porcentaje_gdp_industria: double, gdp_usd: double, gdp_percapita: double]

In [0]:
spark_ec.write.mode('overwrite').option("mergeSchema", "true").saveAsTable('transformed_data.eje_economico')

### Eje Ambiental

Fuente Bano mundial de datos

In [0]:
dataAmb = {'ER.H2O.INTR.PC':'Agua_dulce_per_cápita_en_m3'}

In [0]:
eje_ambiental = getDataFrameDatabank(countries_list, dataAmb)

dimention of data list: Agua_dulce_per_cápita_en_m3
2046


In [0]:
spark_amb = spark.createDataFrame(eje_ambiental)

In [0]:
spark_amb.write.mode('overwrite').saveAsTable('transformed_data.eje_ambiental')

Otras fuentes

In [0]:
# File location and type
file_location = "/FileStore/tables/Epi_des_nat.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

In [0]:
eje_ambiental_pais = df.toPandas()
eje_ambiental_pais.sort_values(by=['country'], inplace = True)
eje_ambiental_pais.reset_index(drop = True, inplace = True)

In [0]:
spark_amb_pais = spark.createDataFrame(eje_ambiental_pais)

In [0]:
spark_amb_pais.write.mode('overwrite').saveAsTable('transformed_data.eje_ambiental_pais')

### Eje Politica

In [0]:
#Aca va el csv que trae Ale

### Eje Educacion

In [0]:
dataEdu = {'SE.ADT.LITR.ZS':'%_de_+15_anios_que_pueden_leer_y_escribir',
            'SE.SEC.CUAT.UP.ZS':'%_de_+25_anios_con_secundario_completo'
           }
            

In [0]:
eje_educacion = getDataFrameDatabank(countries_list, dataEdu)

dimention of data list: %_de_+15_anios_que_pueden_leer_y_escribir
2046
dimention of data list: %_de_+25_anios_con_secundario_completo
2046
is equal times
is equal countries


In [0]:
spark_ed = spark.createDataFrame(eje_educacion)
spark_ed


Out[26]: DataFrame[country: string, year: string, %_de_+15_anios_que_pueden_leer_y_escribir: double, %_de_+25_anios_con_secundario_completo: double]

In [0]:
spark_ed.write.mode('overwrite').option("mergeSchema", "true").saveAsTable('transformed_data.eje_educacion')